# Configurações Iniciais

In [32]:
# Importando bibliotecas
!pip install xgboost
import numpy as np
from numpy import mean, std
import pandas as pd
import seaborn as sns
import tkinter
import plotly.express as px
from collections import Counter
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import *
from sklearn.feature_selection import RFECV
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor

In [33]:
# Retirar os limitadores de coluna
pd.set_option('display.max_columns', None)

# Unindo as bases de dados e removendo as não utilizadas
df_2020 = pd.read_csv("base_inteli_2022_2023.csv", ";")
df_2022 = pd.read_csv("base_inteli_2022_2023.csv", ';')
df_all = pd.concat([df_2020, df_2022])

C:\Users\Inteli\AppData\Local\Temp\ipykernel_21012\4002162432.py:5: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_2020 = pd.read_csv("base_inteli_2022_2023.csv", ";")
C:\Users\Inteli\AppData\Local\Temp\ipykernel_21012\4002162432.py:6: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_2022 = pd.read_csv("base_inteli_2022_2023.csv", ';')


# Exploração dos dados


###Identificação das colunas

A identificação das colunas desempenha um papel importante no processo de exploração. Através dela, podemos conhecer mais sobre os tipos de dados presente em cada coluna e direcionar o tratamento adequado que iremos utilizar para cada caso.

In [34]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 866508 entries, 0 to 433253
Data columns (total 26 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   date                          866508 non-null  object 
 1   weekday_name                  866508 non-null  object 
 2   sku                           866508 non-null  object 
 3   unit_price                    866508 non-null  float64
 4   mobly_item                    866508 non-null  object 
 5   shipment_type                 866508 non-null  object 
 6   anchor_category               866508 non-null  object 
 7   product_department            866508 non-null  object 
 8   product_category              866508 non-null  object 
 9   origin_country                866508 non-null  object 
 10  process_costing               866508 non-null  object 
 11  sku_color                     866508 non-null  object 
 12  sku_height                    813492 non-nul

Através do método `info()`, foi possível classificar nossas características em numéricas ou categóricas:

**Colunas numéricas**

---

* unit_price
* sku_height
* sku_width
* sku_length
* sku_weight
* winning_price
* revenue
* items_sold
* avg_website_visits_last_week
* supplier_delivery_time
* stock_qty
* revenue_bundle
* items_sold_bundle
<br>
<br>

**Colunas categóricas**

---



* date
* weekday_name
* sku
* mobly_item
* shipment_type
* anchor_category
* product_department
* product_category
* origin_country
* process_costing
* sku_color
* price_status
* flag_bundle

###Estatística descritiva das colunas



A estatística descritiva é uma parte da estatística que envolve a coleta, organização, resumo e interpretação de dados para descrever características importantes de um conjunto de informações. Ela se concentra em transformar dados brutos em informações compreensíveis e significativas, permitindo uma compreensão mais clara das tendências, padrões e distribuições presentes nos dados.

Para o caso do nosso projeto, a estatística descritiva desempenha um papel importante para a ideação de um modelo preditivo de regressão. Antes de construir esse modelo, é fundamental entender a distribuição dos dados históricos de vendas. Então, a estatística descritiva, como a média, mediana, desvio padrão e percentis, ajuda a identificar não apenas as tendências, mas também os outliers nos dados de vendas passados.

O método nativo `describe()` da biblioteca Pandas foi utilizado para fazer a análise exploratória dos dados, já que fornece estatísticas para cada uma das colunas do dataframe.

In [35]:
df_all.describe()

,unit_price,sku_height,sku_width,sku_length,sku_weight,winning_price,revenue,items_sold,avg_website_visits_last_week,supplier_delivery_time,stock_qty,revenue_bundle,items_sold_bundle
count,866508.000000,813492.000000,813492.000000,813492.000000,813492.000000,216126.000000,866508.000000,866508.000000,866508.000000,866508.000000,866508.000000,866508.000000,866508.000000
mean,1160.022360,37.430504,73.234059,145.586652,58.008478,929.663083,1646.886429,2.111570,80.566465,23.571245,60.187077,102.839263,0.670110
std,837.686073,24.357225,27.773830,60.479803,72.915035,729.047487,3697.123494,4.564029,118.553581,27.406661,147.116738,658.052235,1.221231
min,69.990000,0.200000,0.800000,1.000000,1.000000,11.310000,-32.660000,0.000000,0.000000,2.000000,-1.000000,0.000000,0.000000
25%,598.820000,13.000000,52.000000,90.000000,16.300000,418.930000,0.000000,0.000000,17.285714,8.000000,0.000000,0.000000,0.000000
50%,909.960000,34.000000,75.000000,153.000000,36.000000,768.620000,538.580000,1.000000,41.857143,11.000000,6.000000,0.000000,1.000000
75%,1499.990000,60.000000,90.000000,189.000000,63.850000,1241.800000,1941.840000,2.000000,95.714286,17.000000,62.000000,0.000000,1.000000
max,9729.990000,152.000000,240.000000,690.000000,553.570000,9399.860000,349834.800000,281.000000,2419.857143,80.000000,3971.000000,60952.660000,164.000000


Com o banco de dados original apresentando algumas informações faltantes em certas linhas, as estatísticas descritivas puderam ser utilizadas para imputar os dados ausentes. Isso pôde ser feito por meio da utilização da mediana, por exemplo, atribuindo o valor dessa tendência central nos campos em branco.

# Pré-processamento


###Limpeza de dados


**Tratamento de missings e remoção das colunas**

---
Removemos as colunas que possuíam somente um valor exclusivo, aquelas que apresentavam mais de 50% dos seus valores vazios ou sem registro e as que continham valores irrelevantes para o nosso modelo.

As colunas possuíam poucos missing values, apenas presentes em algumas dimensões de sku’s, então optamos por substituir esses valores por 0, visto que a inexistência de informações sobre suas dimensões não alterava em sua venda.

In [36]:
# Função para encontrar o índice da primeira venda ou visita
def find_first_occurrence(df_all):
    condition = (df_all['items_sold'] > 0) | (df_all['avg_website_visits_last_week'] > 0)
    if condition.any():
        return condition.idxmax()
    else:
        return None

# Aplica a função para cada grupo de Produto e cria a máscara
mask = df_all.groupby('sku').apply(find_first_occurrence).reset_index(name='first_occurrence_index')
df_all = df_all.merge(mask, on='sku', how='left')
keep_rows = df_all.index >= df_all['first_occurrence_index']

# Filtrar o dataframe usando a máscara
df_all = df_all[keep_rows].drop(columns=['first_occurrence_index'])

display(df_all.sort_values('date'))

,date,weekday_name,sku,unit_price,mobly_item,shipment_type,anchor_category,product_department,product_category,origin_country,process_costing,sku_color,sku_height,sku_width,sku_length,sku_weight,price_status,winning_price,revenue,items_sold,avg_website_visits_last_week,supplier_delivery_time,stock_qty,flag_bundle,revenue_bundle,items_sold_bundle
0,2022-01-01,Saturday,RO870CL89XVAMOB-709898,376.99,mobly item,crossdocking,Kitchen Cabinets,Móveis,Cozinha,Nacional,no,branco,10.0,45.0,125.0,33.50,Sem Registro,NaN,389.55,1,76.857143,5,37,SKU vendido em conjunto ou sozinho,0.0,1
433502,2022-01-01,Saturday,NE528TA31BMMMOB-729552,1835.99,mobly item,crossdocking,Dining Sets,Móveis,Sala de Jantar,Nacional,no,ipê,0.2,0.8,1.0,60.00,Sem Registro,NaN,0.00,0,18.714286,6,0,SKU vendido somente sozinho,0.0,0
433501,2022-01-01,Saturday,LE761CH92EADMOB-850577,1799.99,mobly item,próprio,Office Chairs,Gamer,Móveis Gamer,Importado,no,preto,33.0,67.0,85.0,22.40,Sem Registro,NaN,0.00,0,3.000000,80,108,SKU vendido somente sozinho,0.0,0
433500,2022-01-01,Saturday,MO173CH74CDXMOB-798616,1029.99,mobly item,próprio,Bar Stools,Keva,Sala de Jantar,Importado,no,marrom,NaN,NaN,NaN,NaN,Sem Registro,NaN,0.00,0,7.833333,80,1,SKU vendido somente sozinho,0.0,0
433499,2022-01-01,Saturday,FR314UP56QMTMOB-360475,1799.99,mobly item,crossdocking,Sofas,Keva,Sala de Estar,Nacional,no,cinza,70.0,89.0,200.0,64.85,Sem Registro,NaN,0.00,0,2.857143,5,0,SKU vendido em conjunto ou sozinho,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432975,2023-07-03,Monday,DE350UP94AFPMOB-592995,969.99,mobly item,crossdocking,Sofas,Móveis,Sala de Estar,Nacional,yes,cinza,60.0,90.0,185.0,40.75,Único Disponível,969.99,0.00,0,38.857143,17,838,SKU vendido em conjunto ou sozinho,0.0,0
432974,2023-07-03,Monday,MO173UP55ZRCMOB-864705,2323.93,mobly item,crossdocking,Sofas,Móveis,Sala de Estar,Nacional,yes,grafite,75.0,89.0,210.0,87.10,Único Disponível,2323.93,0.00,0,46.285714,17,2,SKU vendido em conjunto ou sozinho,0.0,0
432973,2023-07-03,Monday,MO173UP08NRNMOB-391634,2007.93,mobly item,crossdocking,Sofas,Móveis,Sala de Estar,Nacional,yes,cinza,56.0,82.0,160.0,125.60,Único Disponível,2007.93,0.00,0,77.142857,17,1,SKU vendido em conjunto ou sozinho,0.0,0
432983,2023-07-03,Monday,SH249TA17HVAMOB-493269,569.96,mobly item,crossdocking,Dining Tables,Móveis,Sala de Jantar,Nacional,no,preto,25.0,94.0,94.0,11.65,Perdendo,190.00,0.00,0,19.571429,2,26,SKU vendido em conjunto ou sozinho,0.0,0


In [37]:
df_fixed = df_all.drop(["mobly_item",'price_status','winning_price', 'revenue','revenue_bundle'], axis ='columns').fillna(0).reset_index(drop = True)
display(df_fixed)

,date,weekday_name,sku,unit_price,shipment_type,anchor_category,product_department,product_category,origin_country,process_costing,sku_color,sku_height,sku_width,sku_length,sku_weight,items_sold,avg_website_visits_last_week,supplier_delivery_time,stock_qty,flag_bundle,items_sold_bundle
0,2022-01-01,Saturday,RO870CL89XVAMOB-709898,376.99,crossdocking,Kitchen Cabinets,Móveis,Cozinha,Nacional,no,branco,10.0,45.0,125.0,33.50,1,76.857143,5,37,SKU vendido em conjunto ou sozinho,1
1,2022-01-01,Saturday,CI381TA53CXAMOB-782057,1156.99,crossdocking,Dining Tables,Móveis,Sala de Jantar,Nacional,no,marrom claro,15.0,130.0,130.0,11.00,0,32.285714,14,1,SKU vendido somente sozinho,0
2,2022-01-01,Saturday,MO173UP38VJHMOB-414464,1519.99,crossdocking,Sofas,Keva,Sala de Estar,Nacional,yes,cinza,65.0,90.0,145.0,42.15,0,70.142857,11,1,SKU vendido em conjunto ou sozinho,0
3,2022-01-01,Saturday,OL487CL08DELMOB-384528,530.99,crossdocking,Dressers and Nightstands,Móveis,Quarto,Nacional,no,carvalho,11.0,48.0,90.0,63.85,0,30.714286,7,193,SKU vendido em conjunto ou sozinho,0
4,2022-01-01,Saturday,RU994TA18APHMOB-575967,1349.99,crossdocking,Dining Sets,Móveis,Sala de Jantar,Nacional,no,imbuia,80.0,90.0,130.0,102.40,0,24.285714,12,1,SKU vendido em conjunto ou sozinho,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866476,2023-07-03,Monday,HR667UP36BSLMOB-988738,1717.95,crossdocking,Sofabeds,Móveis,Sala de Estar,Nacional,no,cinza,50.0,115.0,200.0,40.00,0,154.142857,10,1,SKU vendido somente sozinho,0
866477,2023-07-03,Monday,DE626CL54BLLMOB-798176,1659.88,crossdocking,Closets,Móveis,Quarto,Nacional,no,marrom,14.0,56.0,230.0,150.00,0,138.000000,15,16,SKU vendido somente sozinho,0
866478,2023-07-03,Monday,IM630CH36ZRVMOB-864724,909.96,crossdocking,Dining Chairs,Móveis,Sala de Jantar,Nacional,no,bege,48.0,54.0,62.0,18.60,0,32.714286,10,20,SKU vendido em conjunto ou sozinho,0
866479,2023-07-03,Monday,MO173CH87TWQMOB-519074,619.98,próprio,Bar Stools,Keva,Sala de Jantar,Importado,no,preto,39.0,48.0,72.0,14.70,0,19.000000,80,136,SKU vendido em conjunto ou sozinho,0


In [38]:
df_fixed['has_stock'] = df_fixed['stock_qty'].apply(lambda x: 'tem' if x > 0 else 'não')
df_fixed = df_fixed.drop('stock_qty', axis = 1)
display(df_fixed)

,date,weekday_name,sku,unit_price,shipment_type,anchor_category,product_department,product_category,origin_country,process_costing,sku_color,sku_height,sku_width,sku_length,sku_weight,items_sold,avg_website_visits_last_week,supplier_delivery_time,flag_bundle,items_sold_bundle,has_stock
0,2022-01-01,Saturday,RO870CL89XVAMOB-709898,376.99,crossdocking,Kitchen Cabinets,Móveis,Cozinha,Nacional,no,branco,10.0,45.0,125.0,33.50,1,76.857143,5,SKU vendido em conjunto ou sozinho,1,tem
1,2022-01-01,Saturday,CI381TA53CXAMOB-782057,1156.99,crossdocking,Dining Tables,Móveis,Sala de Jantar,Nacional,no,marrom claro,15.0,130.0,130.0,11.00,0,32.285714,14,SKU vendido somente sozinho,0,tem
2,2022-01-01,Saturday,MO173UP38VJHMOB-414464,1519.99,crossdocking,Sofas,Keva,Sala de Estar,Nacional,yes,cinza,65.0,90.0,145.0,42.15,0,70.142857,11,SKU vendido em conjunto ou sozinho,0,tem
3,2022-01-01,Saturday,OL487CL08DELMOB-384528,530.99,crossdocking,Dressers and Nightstands,Móveis,Quarto,Nacional,no,carvalho,11.0,48.0,90.0,63.85,0,30.714286,7,SKU vendido em conjunto ou sozinho,0,tem
4,2022-01-01,Saturday,RU994TA18APHMOB-575967,1349.99,crossdocking,Dining Sets,Móveis,Sala de Jantar,Nacional,no,imbuia,80.0,90.0,130.0,102.40,0,24.285714,12,SKU vendido em conjunto ou sozinho,0,tem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866476,2023-07-03,Monday,HR667UP36BSLMOB-988738,1717.95,crossdocking,Sofabeds,Móveis,Sala de Estar,Nacional,no,cinza,50.0,115.0,200.0,40.00,0,154.142857,10,SKU vendido somente sozinho,0,tem
866477,2023-07-03,Monday,DE626CL54BLLMOB-798176,1659.88,crossdocking,Closets,Móveis,Quarto,Nacional,no,marrom,14.0,56.0,230.0,150.00,0,138.000000,15,SKU vendido somente sozinho,0,tem
866478,2023-07-03,Monday,IM630CH36ZRVMOB-864724,909.96,crossdocking,Dining Chairs,Móveis,Sala de Jantar,Nacional,no,bege,48.0,54.0,62.0,18.60,0,32.714286,10,SKU vendido em conjunto ou sozinho,0,tem
866479,2023-07-03,Monday,MO173CH87TWQMOB-519074,619.98,próprio,Bar Stools,Keva,Sala de Jantar,Importado,no,preto,39.0,48.0,72.0,14.70,0,19.000000,80,SKU vendido em conjunto ou sozinho,0,tem


In [39]:
df_fixed[df_fixed['items_sold'] != -1].reset_index(drop=True, inplace=True)

In [40]:
df_fixed

,date,weekday_name,sku,unit_price,shipment_type,anchor_category,product_department,product_category,origin_country,process_costing,sku_color,sku_height,sku_width,sku_length,sku_weight,items_sold,avg_website_visits_last_week,supplier_delivery_time,flag_bundle,items_sold_bundle,has_stock
0,2022-01-01,Saturday,RO870CL89XVAMOB-709898,376.99,crossdocking,Kitchen Cabinets,Móveis,Cozinha,Nacional,no,branco,10.0,45.0,125.0,33.50,1,76.857143,5,SKU vendido em conjunto ou sozinho,1,tem
1,2022-01-01,Saturday,CI381TA53CXAMOB-782057,1156.99,crossdocking,Dining Tables,Móveis,Sala de Jantar,Nacional,no,marrom claro,15.0,130.0,130.0,11.00,0,32.285714,14,SKU vendido somente sozinho,0,tem
2,2022-01-01,Saturday,MO173UP38VJHMOB-414464,1519.99,crossdocking,Sofas,Keva,Sala de Estar,Nacional,yes,cinza,65.0,90.0,145.0,42.15,0,70.142857,11,SKU vendido em conjunto ou sozinho,0,tem
3,2022-01-01,Saturday,OL487CL08DELMOB-384528,530.99,crossdocking,Dressers and Nightstands,Móveis,Quarto,Nacional,no,carvalho,11.0,48.0,90.0,63.85,0,30.714286,7,SKU vendido em conjunto ou sozinho,0,tem
4,2022-01-01,Saturday,RU994TA18APHMOB-575967,1349.99,crossdocking,Dining Sets,Móveis,Sala de Jantar,Nacional,no,imbuia,80.0,90.0,130.0,102.40,0,24.285714,12,SKU vendido em conjunto ou sozinho,0,tem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866476,2023-07-03,Monday,HR667UP36BSLMOB-988738,1717.95,crossdocking,Sofabeds,Móveis,Sala de Estar,Nacional,no,cinza,50.0,115.0,200.0,40.00,0,154.142857,10,SKU vendido somente sozinho,0,tem
866477,2023-07-03,Monday,DE626CL54BLLMOB-798176,1659.88,crossdocking,Closets,Móveis,Quarto,Nacional,no,marrom,14.0,56.0,230.0,150.00,0,138.000000,15,SKU vendido somente sozinho,0,tem
866478,2023-07-03,Monday,IM630CH36ZRVMOB-864724,909.96,crossdocking,Dining Chairs,Móveis,Sala de Jantar,Nacional,no,bege,48.0,54.0,62.0,18.60,0,32.714286,10,SKU vendido em conjunto ou sozinho,0,tem
866479,2023-07-03,Monday,MO173CH87TWQMOB-519074,619.98,próprio,Bar Stools,Keva,Sala de Jantar,Importado,no,preto,39.0,48.0,72.0,14.70,0,19.000000,80,SKU vendido em conjunto ou sozinho,0,tem


### Adequação de variáveis numéricas e categóricas


**Separação das variáveis**

---
*(Revisar e corrigir)*<BR>
Todas as colunas identificadas como categóricas em nossa database foram percorridas, contando o número de valores únicos presentes. Com base nessa contagem, foi possível compreender a variedade de categorias por atributo, separando as colunas pelo método que seriam codificadas: One-Hot Encoder ou Label Encoder.

In [41]:
columns = list(df_fixed.columns)

categoric_columns = []
numeric_columns = []

for i in columns:
    if len(df_fixed[i].unique()) > 7:
        numeric_columns.append(i)
    else:
        categoric_columns.append(i)



print('Label Encoding: ',numeric_columns)
print('One-Hot Encoding: ',categoric_columns)

Label Encoding:  ['date', 'sku', 'unit_price', 'anchor_category', 'product_category', 'sku_color', 'sku_height', 'sku_width', 'sku_length', 'sku_weight', 'items_sold', 'avg_website_visits_last_week', 'supplier_delivery_time', 'items_sold_bundle']
One-Hot Encoding:  ['weekday_name', 'shipment_type', 'product_department', 'origin_country', 'process_costing', 'flag_bundle', 'has_stock']


**One-Hot Encoder**


---

Empregamos o One-Hot Encoder em colunas categóricas com até 7 elementos. Essa escolha foi feita para converter essas colunas em um formato binário sem sobrecarregar o sistema. O processo desse codificador gera uma coluna nova para representar cada categoria, na qual o valor é atribuído como "1" ou "0" para indicar se está presente ou ausente, respectivamente.





In [42]:
# Selecionando as colunas para sofrerem codificação
columns_to_encode = ['shipment_type', 'product_department', 'origin_country', 'process_costing', 'flag_bundle']
data_to_encode = df_fixed[columns_to_encode]

# Transformando as colunas para um novo DataFrame
encoder = OneHotEncoder(sparse=False)
encoded = encoder.fit_transform(data_to_encode)
df_oneHot = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(columns_to_encode))

# Substituir as colunas originais pelas colunas do df_oneHot
df_fixed_encoded = df_fixed.drop(columns=columns_to_encode)  # Remover as colunas originais
df_fixed_encoded = pd.concat([df_fixed_encoded, df_oneHot], axis=1)  # Concatenar as colunas codificadas

display(df_fixed_encoded)

c:\Users\Inteli\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,date,weekday_name,sku,unit_price,anchor_category,product_category,sku_color,sku_height,sku_width,sku_length,sku_weight,items_sold,avg_website_visits_last_week,supplier_delivery_time,items_sold_bundle,has_stock,shipment_type_crossdocking,shipment_type_próprio,product_department_Cama e Banho,product_department_Decoração,product_department_Gamer,product_department_Infantil,product_department_Keva,product_department_Móveis,origin_country_Importado,origin_country_Nacional,process_costing_no,process_costing_yes,flag_bundle_SKU vendido em conjunto ou sozinho,flag_bundle_SKU vendido somente sozinho
0,2022-01-01,Saturday,RO870CL89XVAMOB-709898,376.99,Kitchen Cabinets,Cozinha,branco,10.0,45.0,125.0,33.50,1,76.857143,5,1,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
1,2022-01-01,Saturday,CI381TA53CXAMOB-782057,1156.99,Dining Tables,Sala de Jantar,marrom claro,15.0,130.0,130.0,11.00,0,32.285714,14,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2,2022-01-01,Saturday,MO173UP38VJHMOB-414464,1519.99,Sofas,Sala de Estar,cinza,65.0,90.0,145.0,42.15,0,70.142857,11,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3,2022-01-01,Saturday,OL487CL08DELMOB-384528,530.99,Dressers and Nightstands,Quarto,carvalho,11.0,48.0,90.0,63.85,0,30.714286,7,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
4,2022-01-01,Saturday,RU994TA18APHMOB-575967,1349.99,Dining Sets,Sala de Jantar,imbuia,80.0,90.0,130.0,102.40,0,24.285714,12,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866476,2023-07-03,Monday,HR667UP36BSLMOB-988738,1717.95,Sofabeds,Sala de Estar,cinza,50.0,115.0,200.0,40.00,0,154.142857,10,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
866477,2023-07-03,Monday,DE626CL54BLLMOB-798176,1659.88,Closets,Quarto,marrom,14.0,56.0,230.0,150.00,0,138.000000,15,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
866478,2023-07-03,Monday,IM630CH36ZRVMOB-864724,909.96,Dining Chairs,Sala de Jantar,bege,48.0,54.0,62.0,18.60,0,32.714286,10,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
866479,2023-07-03,Monday,MO173CH87TWQMOB-519074,619.98,Bar Stools,Sala de Jantar,preto,39.0,48.0,72.0,14.70,0,19.000000,80,0,tem,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


**Label Encoder**


---

Utilizamos o Label Encoder em colunas categóricas com 7 ou mais elementos. Nesse processo, cada item da categoria era atribuído a um número inteiro distinto, não expandindo o número de colunas.




In [43]:
# Agrupa os items vendidos de acordo com cada item de uma coluna
color_sums = df_fixed_encoded.groupby(['sku_color'])['items_sold'].sum().sort_values(ascending=True)
sku_sums = df_fixed_encoded.groupby(['sku'])['items_sold'].sum().sort_values(ascending=True)
anchor_sums = df_fixed_encoded.groupby(['anchor_category'])['items_sold'].sum().sort_values(ascending=True)
product_sums = df_fixed_encoded.groupby(['product_category'])['items_sold'].sum().sort_values(ascending=True)

# Cria um dicionário mapeando as cores de forma ordinal (Maior para menor)
color_ordinal_mapping = {color: i for i, color in enumerate(color_sums.index)}
sku_ordinal_mapping = {sku: i for i, sku in enumerate(sku_sums.index)}
anchor_ordinal_mapping = {anchor: i for i, anchor in enumerate(anchor_sums.index)}
product_ordinal_mapping = {product: i for i, product in enumerate(product_sums.index)}

# Aplica Ordinal encoding
encoder_color = OrdinalEncoder(categories=[list(color_ordinal_mapping.keys())])
encoder_sku = OrdinalEncoder(categories=[list(sku_ordinal_mapping.keys())])
encoder_anchor = OrdinalEncoder(categories=[list(anchor_ordinal_mapping.keys())])
encoder_product = OrdinalEncoder(categories=[list(product_ordinal_mapping.keys())])

# Cria as novas colunas
df_fixed_encoded['color_encoded'] = encoder_color.fit_transform(df_fixed_encoded[['sku_color']])
df_fixed_encoded['sku_encoded'] = encoder_sku.fit_transform(df_fixed_encoded[['sku']])
df_fixed_encoded['anchor_category_encoded'] = encoder_anchor.fit_transform(df_fixed_encoded[['anchor_category']])
df_fixed_encoded['product_category_encoded'] = encoder_product.fit_transform(df_fixed_encoded[['product_category']])

df_encoded = df_fixed_encoded.drop(['sku','anchor_category','product_category','sku_color'], axis = 1)
df_test = df_fixed_encoded.drop(['sku','anchor_category','product_category','sku_color'], axis = 1)
display(df_encoded)

,date,weekday_name,unit_price,sku_height,sku_width,sku_length,sku_weight,items_sold,avg_website_visits_last_week,supplier_delivery_time,items_sold_bundle,has_stock,shipment_type_crossdocking,shipment_type_próprio,product_department_Cama e Banho,product_department_Decoração,product_department_Gamer,product_department_Infantil,product_department_Keva,product_department_Móveis,origin_country_Importado,origin_country_Nacional,process_costing_no,process_costing_yes,flag_bundle_SKU vendido em conjunto ou sozinho,flag_bundle_SKU vendido somente sozinho,color_encoded,sku_encoded,anchor_category_encoded,product_category_encoded
0,2022-01-01,Saturday,376.99,10.0,45.0,125.0,33.50,1,76.857143,5,1,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,37.0,717.0,15.0,11.0
1,2022-01-01,Saturday,1156.99,15.0,130.0,130.0,11.00,0,32.285714,14,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,25.0,227.0,13.0,12.0
2,2022-01-01,Saturday,1519.99,65.0,90.0,145.0,42.15,0,70.142857,11,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,36.0,539.0,27.0,15.0
3,2022-01-01,Saturday,530.99,11.0,48.0,90.0,63.85,0,30.714286,7,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,33.0,467.0,18.0,14.0
4,2022-01-01,Saturday,1349.99,80.0,90.0,130.0,102.40,0,24.285714,12,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,27.0,147.0,14.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866476,2023-07-03,Monday,1717.95,50.0,115.0,200.0,40.00,0,154.142857,10,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,36.0,206.0,21.0,15.0
866477,2023-07-03,Monday,1659.88,14.0,56.0,230.0,150.00,0,138.000000,15,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,35.0,569.0,25.0,14.0
866478,2023-07-03,Monday,909.96,48.0,54.0,62.0,18.60,0,32.714286,10,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,31.0,613.0,16.0,12.0
866479,2023-07-03,Monday,619.98,39.0,48.0,72.0,14.70,0,19.000000,80,0,tem,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,38.0,380.0,19.0,12.0


**Tratamento de datas**


---
A coluna "date" possui uma alta relevância para nosso modelo, porém necessitava ser modificada para se adaptar melhor ao funcionamento dele.

Transformamos seu formato de Ano / Mês / Dia para uma divisão em diversas colunas representando. Assim, o modelo consegue ler melhor os dados de data uma vez que eles foram passados de String > DateTime > Números, facilitando o entendimento da máquina e do modelo.


In [44]:
# Transformar os valores de "String" de data em "DateTime"
df_encoded['date'] = pd.to_datetime(df_encoded['date'], format='%Y-%m-%d')

# Extraindo os componentes da date
df_encoded['ano'] = df_encoded['date'].dt.year
df_encoded['mes'] = df_encoded['date'].dt.month
df_encoded['dia'] = df_encoded['date'].dt.day
df_encoded['dia_da_semana'] = df_encoded['date'].dt.weekday  # 0: Segunda-feira, 1: Terça-feira, etc.
df_encoded['semana_do_ano'] = df_encoded['date'].dt.isocalendar().week
df_encoded['trimestre'] = df_encoded['date'].dt.quarter
df_encoded['dia_do_ano'] = df_encoded['date'].dt.dayofyear
df_encoded['eh_fim_de_semana'] = df_encoded['date'].dt.weekday >= 5  # Retorna True para sábado e domingo

# Remoção das colunas antigas
df_dates_fixed = df_encoded.drop(['weekday_name'], axis = 1)
display(df_dates_fixed)

,date,unit_price,sku_height,sku_width,sku_length,sku_weight,items_sold,avg_website_visits_last_week,supplier_delivery_time,items_sold_bundle,has_stock,shipment_type_crossdocking,shipment_type_próprio,product_department_Cama e Banho,product_department_Decoração,product_department_Gamer,product_department_Infantil,product_department_Keva,product_department_Móveis,origin_country_Importado,origin_country_Nacional,process_costing_no,process_costing_yes,flag_bundle_SKU vendido em conjunto ou sozinho,flag_bundle_SKU vendido somente sozinho,color_encoded,sku_encoded,anchor_category_encoded,product_category_encoded,ano,mes,dia,dia_da_semana,semana_do_ano,trimestre,dia_do_ano,eh_fim_de_semana
0,2022-01-01,376.99,10.0,45.0,125.0,33.50,1,76.857143,5,1,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,37.0,717.0,15.0,11.0,2022,1,1,5,52,1,1,True
1,2022-01-01,1156.99,15.0,130.0,130.0,11.00,0,32.285714,14,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,25.0,227.0,13.0,12.0,2022,1,1,5,52,1,1,True
2,2022-01-01,1519.99,65.0,90.0,145.0,42.15,0,70.142857,11,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,36.0,539.0,27.0,15.0,2022,1,1,5,52,1,1,True
3,2022-01-01,530.99,11.0,48.0,90.0,63.85,0,30.714286,7,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,33.0,467.0,18.0,14.0,2022,1,1,5,52,1,1,True
4,2022-01-01,1349.99,80.0,90.0,130.0,102.40,0,24.285714,12,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,27.0,147.0,14.0,12.0,2022,1,1,5,52,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866476,2023-07-03,1717.95,50.0,115.0,200.0,40.00,0,154.142857,10,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,36.0,206.0,21.0,15.0,2023,7,3,0,27,3,184,False
866477,2023-07-03,1659.88,14.0,56.0,230.0,150.00,0,138.000000,15,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,35.0,569.0,25.0,14.0,2023,7,3,0,27,3,184,False
866478,2023-07-03,909.96,48.0,54.0,62.0,18.60,0,32.714286,10,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,31.0,613.0,16.0,12.0,2023,7,3,0,27,3,184,False
866479,2023-07-03,619.98,39.0,48.0,72.0,14.70,0,19.000000,80,0,tem,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,38.0,380.0,19.0,12.0,2023,7,3,0,27,3,184,False


In [45]:
df_final = df_dates_fixed.drop(df_dates_fixed[(df_dates_fixed['semana_do_ano'] >= 46) & (df_dates_fixed['semana_do_ano'] <= 51)].index)
df_final = df_final.reset_index(drop = True)
df_final

,date,unit_price,sku_height,sku_width,sku_length,sku_weight,items_sold,avg_website_visits_last_week,supplier_delivery_time,items_sold_bundle,has_stock,shipment_type_crossdocking,shipment_type_próprio,product_department_Cama e Banho,product_department_Decoração,product_department_Gamer,product_department_Infantil,product_department_Keva,product_department_Móveis,origin_country_Importado,origin_country_Nacional,process_costing_no,process_costing_yes,flag_bundle_SKU vendido em conjunto ou sozinho,flag_bundle_SKU vendido somente sozinho,color_encoded,sku_encoded,anchor_category_encoded,product_category_encoded,ano,mes,dia,dia_da_semana,semana_do_ano,trimestre,dia_do_ano,eh_fim_de_semana
0,2022-01-01,376.99,10.0,45.0,125.0,33.50,1,76.857143,5,1,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,37.0,717.0,15.0,11.0,2022,1,1,5,52,1,1,True
1,2022-01-01,1156.99,15.0,130.0,130.0,11.00,0,32.285714,14,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,25.0,227.0,13.0,12.0,2022,1,1,5,52,1,1,True
2,2022-01-01,1519.99,65.0,90.0,145.0,42.15,0,70.142857,11,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,36.0,539.0,27.0,15.0,2022,1,1,5,52,1,1,True
3,2022-01-01,530.99,11.0,48.0,90.0,63.85,0,30.714286,7,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,33.0,467.0,18.0,14.0,2022,1,1,5,52,1,1,True
4,2022-01-01,1349.99,80.0,90.0,130.0,102.40,0,24.285714,12,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,27.0,147.0,14.0,12.0,2022,1,1,5,52,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798204,2023-07-03,1717.95,50.0,115.0,200.0,40.00,0,154.142857,10,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,36.0,206.0,21.0,15.0,2023,7,3,0,27,3,184,False
798205,2023-07-03,1659.88,14.0,56.0,230.0,150.00,0,138.000000,15,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,35.0,569.0,25.0,14.0,2023,7,3,0,27,3,184,False
798206,2023-07-03,909.96,48.0,54.0,62.0,18.60,0,32.714286,10,0,tem,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,31.0,613.0,16.0,12.0,2023,7,3,0,27,3,184,False
798207,2023-07-03,619.98,39.0,48.0,72.0,14.70,0,19.000000,80,0,tem,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,38.0,380.0,19.0,12.0,2023,7,3,0,27,3,184,False


#Ensembles

In [46]:
df_final['semana_do_ano'] = df_final['semana_do_ano'].astype('int64')

In [47]:
from sklearn.model_selection import StratifiedShuffleSplit

X = df_final
y = df_final['sku_encoded']

# Criar um objeto StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0)

# Obter os índices para a divisão
for indice_a, indice_b in sss.split(X, y):
    dados_a = df_final.iloc[indice_a]
    dados_b = df_final.iloc[indice_b]


Usando dados_a

colocar o cod da seleção das features

In [48]:
featuresModels = ['dia', 'supplier_delivery_time', 'stock_qty', 'sku_weight', 'semana_do_ano', 'unit_price', 'dia_do_ano', 'sku_length', 'ano', 'dia_da_semana', 'avg_website_visits_last_week', 'sku_encoded', 'color_encoded']

In [49]:
dados_a.reset_index(inplace=True, drop=True)

In [50]:
#Criando um dataframe a partir do df_final, agora readaptado para as features selecionadas
df_parameters = dados_a.drop(df_final.columns.difference(featuresModels), 1)

# Separando as variáveis independentes e dependentes (X e Y)
X_a = df_parameters
y_a = dados_a['items_sold']
# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X_a_train, X_a_test, y_a_train, y_a_test = train_test_split(X_a, y_a, test_size=0.2, random_state=95)

C:\Users\Inteli\AppData\Local\Temp\ipykernel_21012\427663631.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_parameters = dados_a.drop(df_final.columns.difference(featuresModels), 1)


In [51]:
# Instanciando o modelo com os hiperparâmetros definidos
xgb_reg = XGBRegressor(random_state=42)
xgb_reg.fit(X_a_train, y_a_train)

rf = RandomForestRegressor(random_state=42)
rf.fit(X_a_train, y_a_train)

gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_a_train, y_a_train)


GradientBoostingRegressor(random_state=42)

In [52]:
xgb_pred = xgb_reg.predict(X_a_test)
rf_pred = rf.predict(X_a_test)
gbr_pred = gbr.predict(X_a_test)

In [53]:
# Combinando as predições
X_test_meta = np.column_stack((xgb_pred, rf_pred, gbr_pred))

In [54]:
# Treinando o modelo meta
meta_model = RandomForestRegressor(random_state=42)
meta_model.fit(X_test_meta, y_a_test)

RandomForestRegressor(random_state=42)

dados_b

In [55]:
dados_b.reset_index(inplace=True, drop=True)

df_parameters = dados_b.drop(df_final.columns.difference(featuresModels), 1)

X_b = df_parameters
y_b = dados_b['items_sold']


C:\Users\Inteli\AppData\Local\Temp\ipykernel_21012\1494890391.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_parameters = dados_b.drop(df_final.columns.difference(featuresModels), 1)


In [56]:
xgb_pred_b = xgb_reg.predict(X_b)
rf_pred_b = rf.predict(X_b)
gbr_pred_b = gbr.predict(X_b)

In [57]:
X_new_meta = np.column_stack((xgb_pred_b, rf_pred_b, gbr_pred_b))

y_new_pred = meta_model.predict(X_new_meta)


In [58]:
from sklearn.metrics import mean_squared_error, mean_squared_error, r2_score

#Métrica do Erro Quadrático Médio
mse = mean_squared_error(y_b, y_new_pred)
print("Mean Squared Error:", mse)

#Métrica da Raiz Quadrada do Erro Médio
RMSE = np.sqrt(mse)
print('RMSE:', RMSE)

#Métrica do Coeficiente de Determinação R² score
r2 = r2_score(y_b, y_new_pred)
print("R2 score:", r2)

Mean Squared Error: 4.161992337008929
RMSE: 2.0400961587652993
R2 score: 0.7733282599513882
